In [1]:
from selenium.webdriver import Firefox, ChromiumEdge, EdgeOptions
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC

In [2]:
import time

In [3]:
import os

In [4]:
from glob import glob

In [5]:
options = EdgeOptions()
prefs = {
   'browser':{
      'show_hub_popup_on_download_start': False
  },
   'user_experience_metrics':{
      'personalization_data_consent_enabled': True
  }
}
options.add_experimental_option('prefs', prefs)

In [6]:
driver = ChromiumEdge(options=options)

In [7]:
driver.implicitly_wait(1)

In [8]:
driver.get('https://digital.feprecisionplus.com/greateasternlife')

In [9]:
WebDriverWait(driver, 60).until(EC.invisibility_of_element(('id', 'loading-indicator-undefined-overlay')))
driver.find_element('id', 'lblchkCategoryId-1-1').click()

In [10]:
WebDriverWait(driver, 60).until(EC.invisibility_of_element(('id', 'loading-indicator-undefined-overlay')))

True

In [11]:
for _ in range(4):
    WebDriverWait(driver, 60).until(EC.invisibility_of_element(('id', 'loading-indicator-undefined-overlay')))
    time.sleep(1)
    download_icons = driver.find_elements('class name', 'fe-download-icon')
    visible_icons = [download_icon for download_icon in download_icons if download_icon.is_displayed()]
    for visible_icon in visible_icons:
        visible_icon.click()
        time.sleep(1)
        driver.find_element('xpath', '/html/body/div[9]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[2]/form/div/a').click()
        time.sleep(0.5)
        driver.find_elements('xpath', '/html/body/div[9]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[2]/form/div/ul/li/a')[-1].click()
        time.sleep(0.5)
        driver.find_element('id', 'btnCSVPriceHistory').click()
        for i in range(20):
            if glob(os.path.join(os.environ['USERPROFILE'], 'Downloads', 'Great*Download*.xlsx')):
                break
            time.sleep(1)
        else:
            raise TimeoutError
        time.sleep(1)
        for source_file in glob(os.path.join(os.environ['USERPROFILE'], 'Downloads', 'Great*Download*.xlsx')):
            destination_file = os.path.join(os.getcwd(), 'data', 'GreatLink', f'{os.path.basename(source_file).split("_")[0]}.xlsx')
            if os.path.exists(destination_file):
                os.remove(destination_file)
            os.rename(source_file, destination_file)
        time.sleep(0.5)
        driver.find_element('xpath', '/html/body/div[9]/div[1]/button').click()
        time.sleep(0.5)
    WebDriverWait(driver, 60).until(EC.invisibility_of_element(('id', 'loading-indicator-undefined-overlay')))
    driver.find_element('xpath', '/html/body/div[6]/div[1]/div[1]/div/div[1]/div[3]/div[4]/table/tfoot/tr/td/div/div/a[4]').click()
    

In [12]:
WebDriverWait(driver, 60).until(EC.invisibility_of_element(('id', 'loading-indicator-undefined-overlay')))
driver.find_element('id', 'lblchkBespokeColumn5-1-2').click()
WebDriverWait(driver, 60).until(EC.invisibility_of_element(('id', 'loading-indicator-undefined-overlay')))

True

In [13]:
WebDriverWait(driver, 60).until(EC.invisibility_of_element(('id', 'loading-indicator-undefined-overlay')))
time.sleep(1)
download_icons = driver.find_elements('class name', 'fe-download-icon')
visible_icons = [download_icon for download_icon in download_icons if download_icon.is_displayed()]
for visible_icon in visible_icons:
    visible_icon.click()
    time.sleep(1)
    driver.find_element('id', 'radio-dividend').click()
    time.sleep(1)
    driver.find_element('xpath', '/html/body/div[9]/div[2]/div[1]/div/div/div[3]/div[1]/div[2]/div/div[2]/span[2]/form/div/a').click()
    time.sleep(1)
    driver.find_elements('xpath', '/html/body/div[9]/div[2]/div[1]/div/div/div[3]/div[1]/div[2]/div/div[2]/span[2]/form/div/ul/li/a')[-1].click()
    time.sleep(1)
    driver.find_element('id', 'btnCSVDividendHistory').click()
    for i in range(20):
        if glob(os.path.join(os.environ['USERPROFILE'], 'Downloads', 'Great*Download*.xlsx')):
            break
        time.sleep(1)
    else:
        raise TimeoutError
    time.sleep(1)
    for source_file in glob(os.path.join(os.environ['USERPROFILE'], 'Downloads', 'Great*Download*.xlsx')):
        destination_file = os.path.join(os.getcwd(), 'data', 'GreatLink', f'{os.path.basename(source_file).split("_")[0]}_Dividends.xlsx')
        if os.path.exists(destination_file):
            os.remove(destination_file)
        os.rename(source_file, destination_file)
    time.sleep(0.5)
    driver.find_element('xpath', '/html/body/div[9]/div[1]/button').click()
    time.sleep(0.5)

In [14]:
driver.quit()